In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from pympc.geometry.polytope import Polytope
from pympc.optimization.pnnls import linear_program
from time import time

random points

In [9]:
n_points = 1000
n_dimensions = 20
points = [np.random.rand(n_dimensions, 1) for i in range(n_points)]
test_point = np.random.rand(n_dimensions, 1)

convex hull and matrix multiplication

In [4]:
hull = ConvexHull(np.hstack(points).T)
A = hull.equations[:,:-1]
b = - hull.equations[:,-1:]
print A.shape

(701345, 10)


In [5]:
tic = time()
print np.max(A.dot(test_point) - b)
print time() - tic

0.443485377356
0.0524070262909


convex hull linear program

In [12]:
def point_in_convex_hull(point, hull_points):
    """
    Checks if the point p is inside the convex hull of the points {v_1, ..., v_n}.
    It solves the linear program
    J^* := max_{a_1, ..., a_n} min({a_1, ..., a_n})
         = - min_{a_1, ..., a_n} max({-a_1, ..., -a_n})
    subject to
    \sum_{i = 1}^n a_i = 1
    \sum_{i = 1}^n a_i v_i = p
    If J^* >= 0 then p \in conv({v_1, ..., v_n}).
    """
        
    # sum of the coefficients equal to one
    n_h = len(hull_points)
    lhs_eq = np.ones((1, n_h))
    rhs_eq = np.ones((1, 1))

    # linear combination of the points of the hull equal to the new point
    lhs_eq = np.vstack((lhs_eq, np.hstack(hull_points)))
    rhs_eq = np.vstack((rhs_eq, point))

    # max function (with slack variable)
    lhs_eq = np.hstack((lhs_eq, np.zeros((lhs_eq.shape[0], 1))))
    lhs_ineq = np.hstack((-np.eye(n_h), np.ones((n_h, 1))))
    rhs_ineq = np.zeros((n_h, 1))

    # cost function
    f = np.zeros((n_h+1, 1))
    f[-1, 0] = 1.

    # solve the linear program
    tic = time()
    sol = linear_program(-f, lhs_ineq, rhs_ineq, lhs_eq, rhs_eq)
    print time() - tic
    return sol.min < 0.

In [13]:
point_in_convex_hull(test_point, points)

9.38059210777
10.5299301147


False

separating hyperplane

In [15]:
eps = 1.e-3
f = np.zeros((n_dimensions+1, 1))
lhs = np.hstack(points).T
lhs = np.hstack((lhs, - np.ones((n_points, 1))))
lhs = np.vstack((lhs, np.hstack((- test_point.T, np.ones((1,1))))))
rhs = -np.ones((n_points + 1, 1))*eps
sol = linear_program(f, lhs, rhs)
tic = time()
print sol.min
print time() - tic

0.0
0.000197172164917


plot if 2d

In [ ]:
p = Polytope(A, b)
p.assemble()

In [ ]:
p.plot(alpha=0.5, facecolor='w')
plt.scatter([point[0,0] for point in points], [point[1,0] for point in points])
plt.scatter(test_point[0,:], test_point[1,:])
plt.show()

In [ ]:
np.max(A.dot(test_point) - b)